Phase 1: Speech To Braille

In [ ]:
#!/usr/bin/env python3

import os
import wave
import pyaudio
import whisper
from autogen_agentchat.agents import AssistantAgent
import os
import wave
import pyaudio
import whisper
from autogen import ConversableAgent  

# Set your OpenAI API key (replace with your actual key)
OPENAI_API_KEY = 'Enter Your OPENAI API Key Here'

# Check if the API key is provided; if not, raise an error.

if not OPENAI_API_KEY:
    raise ValueError("Error: OPENAI_API_KEY not found in environment variables.")
#!/usr/bin/env python3

# Create a ConversableAgent instance with the desired configuration.
agent = ConversableAgent(
    name="BrailleChatbot",
    llm_config={
        "config_list": [{"model": "gpt-4o-mini", "api_key": OPENAI_API_KEY}]
    },
    code_execution_config=False,  
    human_input_mode="NEVER",  
)

# Define a dictionary mapping regular characters to their Braille equivalents.

BRAILLE_DICT = {
    'a': '⠁', 'b': '⠃', 'c': '⠉', 'd': '⠙', 'e': '⠑',
    'f': '⠋', 'g': '⠛', 'h': '⠓', 'i': '⠊', 'j': '⠚',
    'k': '⠅', 'l': '⠇', 'm': '⠍', 'n': '⠝', 'o': '⠕',
    'p': '⠏', 'q': '⠟', 'r': '⠗', 's': '⠎', 't': '⠞',
    'u': '⠥', 'v': '⠧', 'w': '⠺', 'x': '⠭', 'y': '⠽',
    'z': '⠵',
    ' ': ' ',
    ',': '⠂', '.': '⠲', '?': '⠦', '!': '⠖', ';': '⠆',
    ':': '⠒', '-': '⠤', '(': '⠣', ')': '⠜', '"': '⠐⠦',
    "'": '⠄',
    '0': '⠼⠚', '1': '⠼⠁', '2': '⠼⠃', '3': '⠼⠉', '4': '⠼⠙',
    '5': '⠼⠑', '6': '⠼⠋', '7': '⠼⠛', '8': '⠼⠓', '9': '⠼⠊'
}

def text_to_braille(text: str) -> str:
    return ''.join(BRAILLE_DICT.get(char, '?') for char in text.lower())

def record_audio(filename="temp.wav", record_secs=5, rate=16000):
    chunk = 1024
    pa = pyaudio.PyAudio()
    stream = pa.open(format=pyaudio.paInt16,
                     channels=1,
                     rate=rate,
                     input=True,
                     frames_per_buffer=chunk)
    
    print("Recording...")

    # Read audio chunks for the duration of record_secs

    frames = [stream.read(chunk) for _ in range(int(rate / chunk * record_secs))]
    print("Finished recording.")

    # Stop and close the audio stream
    stream.stop_stream()
    stream.close()
    pa.terminate()
    
    # Save the recorded frames as a WAV file
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(pa.get_sample_size(pyaudio.paInt16))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))

# Define the temporary WAV file name
def main():
    wav_file = "temp.wav"
    record_audio(filename=wav_file, record_secs=8)

    # Load the Whisper model for transcription
    print("Loading Whisper model...")
    whisper_model = whisper.load_model("small")

    # Transcribe the recorded audio
    print("Transcribing audio...")
    result = whisper_model.transcribe(wav_file)
    user_text = result["text"].strip()
    print(f"User said: {user_text}")

    # Remove the temporary audio file
    os.remove(wav_file)

    # Generate AI response using the ConversableAgent
    print("AI thinking...")
    ai_response = agent.generate_reply(messages=[{"role": "user", "content": user_text}])

    # Fallback response if no AI response was generated
    ai_text = ai_response if ai_response else "I couldn't generate a response."
    print(f"AI Response: {ai_text}")

    # Convert the AI response to Braille representation
    braille_output = text_to_braille(ai_text)
    print(f"Braille Output: {braille_output}")

    # Save the Braille output to a text file
    with open("braille_output.txt", "w", encoding="utf-8") as f:
        f.write(braille_output)

    print("Conversation Turn Complete!")

if __name__ == "__main__":
    main()

Phase 2: ASL To Text

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

Training

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

def train_asl_model(
    train_dir="asl_alphabet_train",
    num_epochs=1,
    batch_size=120,
    lr=0.001,
    model_path="asl_model.pth"
):


    # Transforms for training images
    train_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # Standard ImageNet normalization
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

    # Load dataset from the train folder
    # Adjust the path to your actual training images if needed
    train_dataset = datasets.ImageFolder(
        "/home/varsh/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1/asl_alphabet_train/asl_alphabet_train",
        transform=train_transforms
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    class_names = train_dataset.classes
    num_classes = len(class_names)
    print(f"Found {num_classes} classes in '{train_dir}': {class_names}")

    # Create ResNet18 with pretrained ImageNet weights
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    
    # Replace final layer for our number of classes
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}]: "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}")

    # Save model weights
    torch.save(model.state_dict(), model_path)
    print(f"Training complete. Model saved to '{model_path}'")

    return model, class_names

def main():
    # Call the training function
    # Adjust the parameters as needed
    model, class_names = train_asl_model(
        train_dir="asl_alphabet_train",
        num_epochs=5,
        batch_size=120,
        lr=0.001,
        model_path="asl_model.pth"
    )
    print("Finished training.")
    print("Classes are:", class_names)

if __name__ == "__main__":
    main()

Testing

In [ ]:
#!/usr/bin/env python3

import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

def predict_image(img_path, model, class_names):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Same transforms as training (resize, normalize)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

    # Load and preprocess the image
    img = Image.open(img_path).convert("RGB")
    img_t = transform(img).unsqueeze(0).to(device)  # shape: [1, 3, 224, 224]

    # Model inference
    model.eval()
    with torch.no_grad():
        outputs = model(img_t)
        _, preds = torch.max(outputs, 1)

    predicted_class = class_names[preds.item()]
    return predicted_class

def test_on_test_folder(test_dir, model, class_names):
    """
    Iterates over all .jpg files in `test_dir`, infers their label,
    and compares to the true label (parsed from filename).
    
    Example filename: "A_test.jpg" -> true label is "A".
    Prints each result and final accuracy.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Gather all .jpg files in the test directory
    test_files = [f for f in os.listdir(test_dir) if f.lower().endswith(".jpg")]
    total = 0
    correct = 0

    for f in test_files:
        img_path = os.path.join(test_dir, f)

        # Parse the true label from filename (before underscore)
        # e.g. "A_test.jpg" -> "A", "del_123.jpg" -> "del"
        true_label = f.split("_")[0]

        # Inference
        predicted_label = predict_image(img_path, model, class_names)

        # Compare
        total += 1
        if predicted_label == true_label:
            correct += 1

        print(f"File: {f} | Pred: {predicted_label} | True: {true_label}")

    # Final accuracy
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"\nTotal Tested: {total}, Correct: {correct}, Accuracy: {accuracy:.2f}%")

def main():
    # 1. Recreate the exact architecture used in training
    #    We'll assume 29 classes: A-Z + 'del', 'nothing', 'space'.
    num_classes = 29
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Create a base ResNet18 with no pretrained weights
    model = models.resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    # 2. Load the trained weights from file
    state_dict = torch.load("asl_model.pth", map_location=device)
    model.load_state_dict(state_dict)
    model.to(device)

    # 3. The same class_names from your training (alphabetical order)
    class_names = [
        'A','B','C','D','E','F','G','H','I','J','K',
        'L','M','N','O','P','Q','R','S','T','U','V',
        'W','X','Y','Z','del','nothing','space'
    ]

    # 4. Test folder with single images named like "A_test.jpg", "B_test.jpg", etc.
    test_dir = "/home/varsh/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1/asl_alphabet_test/asl_alphabet_test"
    test_on_test_folder(test_dir, model, class_names)

if __name__ == "__main__":
    main()

Demo

In [ ]:
#!/usr/bin/env python3

import cv2
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

def live_demo(model, class_names):
    """
    Real-time ASL sign recognition with a small bounding box (ROI).
    The user should place their hand inside the box.
    Press ESC to exit.
    """

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()

    # Same transforms used during training (minus random augmentation)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

    # Define bounding box (ROI) coordinates and size
    roi_x, roi_y = 50, 50   
    roi_w, roi_h = 224, 224 

    cap = cv2.VideoCapture(0)  
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Flip horizontally for a mirror-like view
        frame = cv2.flip(frame, 1)

        # Draw the bounding box on the frame
        cv2.rectangle(
            frame,
            (roi_x, roi_y),
            (roi_x + roi_w, roi_y + roi_h),
            (0, 255, 0), 2
        )

        # Crop the region of interest
        roi_frame = frame[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]

        # Convert ROI to RGB (OpenCV is BGR) and then to PIL
        roi_rgb = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(roi_rgb)

        # Transform and infer
        img_t = transform(pil_img).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = model(img_t)
            _, preds = torch.max(outputs, 1)

        predicted_label = class_names[preds.item()]

        # Display the predicted label near the ROI
        text_pos = (roi_x, roi_y - 10)  # slightly above the box
        cv2.putText(frame, f"Pred: {predicted_label}",
                    text_pos,
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0, 255, 0), 2)

        # Show the frame
        cv2.imshow("ASL Live Demo (ROI)", frame)

        # Press ESC to quit
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

def main():
    # Same class names as in training
    class_names = [
        'A','B','C','D','E','F','G','H','I','J','K',
        'L','M','N','O','P','Q','R','S','T','U','V',
        'W','X','Y','Z','del','nothing','space'
    ]
    num_classes = len(class_names)

    # Load model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = models.resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    # Load your trained weights
    checkpoint = torch.load("asl_model.pth", map_location=device)
    model.load_state_dict(checkpoint)
    model.to(device)

    # Run the live ROI demo
    live_demo(model, class_names)

if __name__ == "__main__":
    main()

Phase 3: Know Your Rights Chatbot

In [ ]:
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import openai

# OpenAI API key
OPENAI_API_KEY = 'Enter Your API Key Here'
openai.api_key = OPENAI_API_KEY

class PDFChatbot:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.vector_db = None
        self.qa_chain = None  
        # Process the PDF: extract text, split it, create a vector database
        self.load_and_vectorize_pdf()
        # Build the QA chain using the vector database
        self.build_qa_chain()  

# Extract text from each page of the Document.
    def extract_text(self):
        text = ""
        with pdfplumber.open(self.pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text
    
# Split the extracted text into smaller chunks using a recursive character splitter.
    def split_text(self, text):
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
        return splitter.split_text(text)

# Create a vector database (Chroma) from the text chunks using OpenAI embeddings.
    def create_vector_db(self, text_chunks):
        embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
        self.vector_db = Chroma.from_texts(text_chunks, embeddings)

# Extract, split, and vectorize the text.
    def load_and_vectorize_pdf(self):
        text = self.extract_text()
        if not text.strip():
            raise ValueError("No text could be extracted from the PDF.")
        text_chunks = self.split_text(text)
        self.create_vector_db(text_chunks)

# Build a RetrievalQA chain using the vector database as the retriever and OpenAI as the LLM.
    def build_qa_chain(self):
        retriever = self.vector_db.as_retriever()
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY),
            chain_type="stuff",
            retriever=retriever
        )

    def ask(self, query):
        full_query = f"""
You are an AI assistant responding to a user query based on a provided document.
Please provide a detailed answer with references from the document.
Always include a legal disclaimer at the end: "Disclaimer: This response is based on the provided document and is for informational purposes only. Please consult a qualified lawyer for legal advice."

User Query: {query}
"""
        return self.qa_chain.run(full_query)

if __name__ == '__main__':
    pdf_path = "Data.pdf"
    chatbot = PDFChatbot(pdf_path)
    
    while True:
        query = input("Enter your query (or type 'exit' to quit): ")
        if query.lower() == "exit":
            break
        response = chatbot.ask(query)
        print("AI Response:", response)
